In [1]:
# Import libraries:
from bs4 import BeautifulSoup
import requests

import re

import pandas as pd

In [2]:
# Load dataset
filepath = r'C:\Users\ADMIN\Desktop\python\project'
crawl = pd.read_csv(filepath + r'\budget_revenue_crawl\the_numbers_crawl_res\the_numbers_crawl.csv', encoding='UTF-8', engine='python')
crawl

,id,title,year,budget,revenue
0,79698,The Lovers,2015,35000000,53899
1,7978,The Wolfman,2010,150000000,142634358
2,10588,The Cat in the Hat,2003,109000000,133818283
3,10214,Son of the Mask,2005,100000000,59918422
4,77953,The Campaign,2012,95000000,104907746
...,...,...,...,...,...
1568,67238,Cavite,2005,7000,71644
1569,72766,Newlyweds,2011,9000,4584
1570,231617,"Signed, Sealed, Delivered",2013,0,0
1571,126186,Shanghai Calling,2012,0,10443


In [3]:
# Get data missing records
crawl_missing = crawl[(crawl['budget'] == 0) | (crawl['revenue'] == 0)]
crawl_missing

,id,title,year,budget,revenue
8,16858,All That Jazz,1979,0,37823676
10,12096,The Pink Panther,2006,0,10878107
11,161795,Deja Vu,1998,0,6321997
15,9433,The Edge,1997,0,34099255
17,293644,Top Cat Begins,2015,0,0
...,...,...,...,...,...
1561,86304,All Superheroes Must Die,2011,20000,0
1564,157185,Tin Can Man,2007,12000,0
1566,286939,Sanctuary: Quite a Conundrum,2012,200000,0
1570,231617,"Signed, Sealed, Delivered",2013,0,0


In [4]:
# Define google crawling function
def google_crawl(id, title, year):
    headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    
    #budget
    budget = 0
    query = (title + ' ' + year + ' budget').replace(' ', '%20')
    url = "https://google.com/search?q={}".format(query)
    res = requests.request("GET", url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    try:
        budget = soup.find("div", {'class': "Z0LcW XcVN5d"}).text
    except:
        pass
    
    #revenue
    revenue = 0
    query = (title + ' ' + year + ' box office').replace(' ', '%20')
    url = "https://google.com/search?q={}".format(query)
    res = requests.request("GET", url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    try:
        revenue = soup.find("div", {'class': "Z0LcW XcVN5d"}).text
    except:
        pass
    
    return {'id': id,
            'title': title,
            'year' : year,
            'budget': budget,
            'revenue': revenue}

In [15]:
# Crawl data
result = pd.DataFrame(columns = ['id', 'title', 'year', 'budget', 'revenue'])
error = []

for i in crawl_missing.index[:10]:
    try:
        result = result.append(google_crawl(crawl_missing.loc[i, 'id'], crawl_missing.loc[i, 'title'] ,str(crawl_missing.loc[i, 'year'])), ignore_index=True)
    except:
        error.append(i)

In [ ]:
error

In [ ]:
result

In [10]:
# Clean data
def transform(string):
    string = string.replace('\xa0', ' ')
    l = string.split(" ")
    value = l[0]
    value = value.replace('.', '')
    value = value.replace(',', '.')
    if "triệu" in string:
        value = float(value) * 100000
    
    return value

result['budget'] = result['budget'].apply(lambda x: transform(x) if x != 0 else x)
result['revenue'] = result['revenue'].apply(lambda x: transform(x) if x != 0 else x)

In [11]:
# Export to csv file
result.to_csv(filepath + r'\budget_revenue_crawl\google_crawl_res\google_crawl.csv', index=False)